# Worldwide Cases - Trabalho Final - A80790

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from qgis.core import *
from qgis.PyQt.QtGui import *
from qgis.PyQt.QtCore import *
from IPython.display import Image


import datetime

In [2]:
QgsApplication.setPrefixPath(r'C:\\OSGeo4W64\\apps\\qgis', True)
qgs = QgsApplication([], True)
qgs.initQgis()
sys.path.append(r'C:\OSGeo4W64\apps\qgis\python\plugins')
project = QgsProject.instance()

In [3]:
path = "ne_10m_admin_0_countries.shp"
map_layer = QgsVectorLayer(path, 'countries', 'ogr')

if not map_layer.isValid():
    print("Failed to load the layer!")
else:
    project.addMapLayer(map_layer)
    print("Sucess")

Sucess


In [4]:
csv_path = "file:transferir"
csv_layer = QgsVectorLayer(csv_path, 'Data', 'delimitedtext')

if not csv_layer.isValid():
    print('Layer failed to load!')
else:
    project.addMapLayer(csv_layer)
    print("Sucess")

Sucess


In [5]:
#df = pd.read_csv("transferir")
df = pd.read_csv("https://raw.githubusercontent.com/datasets/covid-19/master/data/worldwide-aggregated.csv")
df = df.set_index('Date')
df.head()

,Confirmed,Recovered,Deaths,Increase rate
Date,,,,
2020-01-22,555,28,17,NaN
2020-01-23,654,30,18,17.837838
2020-01-24,941,36,26,43.883792
2020-01-25,1434,39,42,52.391073
2020-01-26,2118,52,56,47.698745


In [6]:
df

,Confirmed,Recovered,Deaths,Increase rate
Date,,,,
2020-01-22,555,28,17,NaN
2020-01-23,654,30,18,17.837838
2020-01-24,941,36,26,43.883792
2020-01-25,1434,39,42,52.391073
2020-01-26,2118,52,56,47.698745
...,...,...,...,...
2020-05-19,4897492,1838995,323285,1.989799
2020-05-20,4996472,1897466,328115,2.021034
2020-05-21,5102424,1948739,332924,2.120536


In [7]:
df_most_recent_date = df.tail(1)

In [8]:
df_most_recent_date

,Confirmed,Recovered,Deaths,Increase rate
Date,,,,
2020-05-23,5310362,2112185,342097,1.910353


In [23]:
import requests

if not os.path.exists('Covid19'):
    os.mkdir("Covid19")
    
r = requests.get(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")  
with open(r'Covid19/time_series_covid19_confirmed_global.csv', 'wb') as f:
    f.write(r.content)
r = requests.get(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")  
with open(r'Covid19/time_series_covid19_deaths_global.csv', 'wb') as f:
    f.write(r.content)
r = requests.get(r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")  
with open(r'Covid19/time_series_covid19_recovered_global.csv', 'wb') as f:
    f.write(r.content)

import urllib.parse
cwd = urllib.parse.quote(os.getcwd() )


uri = r"file:///{}/Covid19/time_series_covid19_confirmed_global.csv?delimiter={}&xField={}&yField={}".format(cwd, ",", "Long", "Lat")
confirmed = QgsVectorLayer(uri, "confirmed", "delimitedtext")
if not confirmed.isValid():
    print("Layer 'confirmed' failed to load!")
else:
    print("Layer 'confirmed' loaded!")

uri = r"file:///{}/Covid19/time_series_covid19_deaths_global.csv?delimiter={}&xField={}&yField={}".format(cwd, ",", "Long", "Lat")
deaths = QgsVectorLayer(uri, "deaths", "delimitedtext")
if not deaths.isValid():
    print("Layer 'deaths' failed to load!")
else: 
    print("Layer 'deaths' loaded!")

uri = r"file:///{}/Covid19/time_series_covid19_recovered_global.csv?delimiter={}&xField={}&yField={}".format(cwd, ",", "Long", "Lat")
recovered = QgsVectorLayer(uri, "recovered", "delimitedtext")
if not recovered.isValid():
    print("Layer 'recovered' failed to load!")
else: 
    sys.path.append('/usr/lib/qgis')
    sys.path.append('/usr/share/qgis/python/plugins')    
    print("Layer 'recovered' loaded!")
    


Layer 'confirmed' loaded!
Layer 'deaths' loaded!
Layer 'recovered' loaded!


In [30]:
from qgis.processing import *
from qgis.analysis import QgsNativeAlgorithms

from processing.core.Processing import Processing

Processing.initialize()

QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())

In [31]:
params = { 
    'INPUT' : confirmed,
    'FIELD' : 'PC',
    'INPUT_2' : deaths, 
    'FIELD_2' : 'PC',
    'FIELDS_TO_COPY' : '',
    'METHOD' : 0,
    'DISCARD_NONMATCHING' : False,
    'PREFIX' : 'deaths_',
    'NON_MATCHING' : 'memory:',
    'OUTPUT' : 'memory:'
}
feedback = QgsProcessingFeedback()

res = processing.run("qgis:joinattributestable", params,feedback=feedback)
comb = res['OUTPUT']

QgsProcessingException: There were errors executing the algorithm.

In [10]:
recoverRate = map_layer.clone()
recoverRate.setName("Taxa de Recuperação")

In [11]:
from qgis.PyQt import QtGui

myOpacity = 1

myTargetField = r' if( "4/15/20"=0,-1,"recovered_4/15/20" / "4/15/20" )'
myRangeList = []

# Make our first symbol and range...
myMin = 0.0
myMax = 0.05
myLabel = '< 5%'
myColour = QtGui.QColor('#000000')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.05
myMax = 0.1
myLabel = '5% - 10%'
myColour = QtGui.QColor('#005500')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.1
myMax = 0.15
myLabel = '10% - 15%'
myColour = QtGui.QColor('#00aa00')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.15
myMax = 0.2
myLabel = '15% - 20%'
myColour = QtGui.QColor('#00ff00')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.2
myMax = 0.25
myLabel = '20% - 25%'
myColour = QtGui.QColor('#44ff44')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.25
myMax = 0.3
myLabel = '25% - 30%'
myColour = QtGui.QColor('#88ff88')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.3
myMax = 0.35
myLabel = '30% - 35%'
myColour = QtGui.QColor('#bbffbb')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = 0.35
myMax = 1
myLabel = '> 35%'
myColour = QtGui.QColor('#ffffff')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)

#now make another symbol and range...
myMin = -1
myMax = -1
myLabel = 'Sem Dados'
myColour = QtGui.QColor('#888888')
mySymbol = QgsSymbol.defaultSymbol(recoverRate.geometryType())
mySymbol.setColor(myColour)
myRange = QgsRendererRange(myMin, myMax, mySymbol, myLabel)
myRangeList.append(myRange)
myRenderer = QgsGraduatedSymbolRenderer('', myRangeList)
myClassificationMethod = QgsApplication.classificationMethodRegistry().method("EqualInterval")
myRenderer.setClassificationMethod(myClassificationMethod)
myRenderer.setClassAttribute(myTargetField)

recoverRate.setRenderer(myRenderer)

In [13]:
xt = map_layer.extent()
# print(xt)
width  = 1200
height = int(width*xt.height()/xt.width())

project = QgsProject().instance()
project.addMapLayers([recoverRate])
manager = project.layoutManager()
layoutName = 'recuperados'
layouts_list = manager.printLayouts()
for layout in layouts_list:
    if layout.name() == layoutName:
        manager.removeLayout(layout)
layout = QgsPrintLayout(project)
layout.initializeDefaults()
layout.setName(layoutName)
manager.addLayout(layout)

pc = layout.pageCollection()
pc.page(0).attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))

map = QgsLayoutItemMap(layout)
map.setRect(1,1,1,1)
map.setLayers([recoverRate])

ms = QgsMapSettings()
ms.setLayers([recoverRate])
ms.setExtent(xt)
map.setExtent(xt)

layout.addLayoutItem(map)

map.attemptMove(QgsLayoutPoint(0,0,QgsUnitTypes.LayoutMillimeters))
map.attemptResize(QgsLayoutSize(width,height,QgsUnitTypes.LayoutMillimeters))

map.setBackgroundColor(QColor(135,206,250))

legend = QgsLayoutItemLegend(layout)
legend.setRect(1,1,1,1)

legend.setScale(4)
legend.setReferencePoint(6)
legend.attemptMove(QgsLayoutPoint(0,height,QgsUnitTypes.LayoutMillimeters))
legend.setLinkedMap(map) 
layout.addItem(legend)
legend.setLegendFilterByMapEnabled(True)

legend.refresh()

In [14]:
exporter = QgsLayoutExporter(manager.layoutByName("recuperados"))

image = exporter.renderPageToImage(0)
image = exporter.renderPageToImage(0)

from IPython.display import Image
imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )